In [0]:
import io
import regex as re
import nltk
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from string import digits

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'

In [4]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
abc = urlopen(link)
zf = ZipFile(io.BytesIO(abc.read()))

In [0]:
spam = pd.read_csv(zf.open('SMSSpamCollection'),sep = '\t',names=["type", "msg"])

In [7]:
spam.head()

,type,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
def toktoktok(text):
  
  text= text.lower()
  text = text.translate(text.maketrans('', '', string.digits))
  text = text.translate(text.maketrans('', '', '''!()-[]{};:'"\,<>./?@#$%^&*_~'''))
  t = word_tokenize(text)
  tokens = ' '.join([word for word in t if word not in stopwords])#removing stop words
  
  return tokens

In [0]:
spam['processed'] = spam["msg"].apply(toktoktok)

In [10]:
spam['processed']

0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry wkly comp win fa cup final tkts st ...
3                     u dun say early hor u c already say
4             nah dont think goes usf lives around though
                              ...                        
5567    nd time tried contact u u £ pound prize claim ...
5568                          ü b going esplanade fr home
5569                          pity mood soany suggestions
5570    guy bitching acted like id interested buying s...
5571                                       rofl true name
Name: processed, Length: 5572, dtype: object

In [0]:

tf = TfidfVectorizer()
X = tf.fit_transform(spam['processed'])

In [0]:
df = pd.DataFrame(X.toarray(), columns=tf.get_feature_names())

In [0]:
y = spam['type']

In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [0]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=42)

In [0]:
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

In [17]:

from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
print(classification_report(y_test,y_pred))
accuracy_score(y_test,y_pred)

              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1448
        spam       1.00      0.80      0.89       224

    accuracy                           0.97      1672
   macro avg       0.98      0.90      0.94      1672
weighted avg       0.97      0.97      0.97      1672



0.9730861244019139

In [0]:
tester = tf.transform(['This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.'])

In [19]:
rfc.fit(X,spam['type']).predict(tester)

array(['spam'], dtype=object)